# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [317]:
import pandas as pd
import numpy as np
import math
import json
import re
#% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# Data Preparation

In [318]:

channels = set()

for label, value in portfolio["channels"].iteritems():
    for i in value:
        channels.add(i)

for channel in channels:
    portfolio[channel] = portfolio["channels"].apply(lambda x: 1 if channel in x else 0)

portfolio = portfolio.drop("channels", axis =1)

portfolio

,reward,difficulty,duration,offer_type,id,web,email,mobile,social
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,0,1,1,1
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,1,0


In [319]:
transcript[transcript["person"] == "78afa995795e4d85b5d9ceeca43f5fef"]

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
15561,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},6
47582,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.89},132
47583,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132
49502,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 17.78},144
53176,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168
85291,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},216
87134,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.67},222
92104,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 29.72},240
141566,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 23.93},378


In [310]:
profile.head()



,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [320]:
def transform_ammount(x):
        
    keys, value = list(x.items())[0]

    if keys == "amount":
        return  value
    else:
        return np.nan
    
def transform_offer_id(x):
        
    keys, value = list(x.items())[0]

    if keys == "offer id":
        return  value
    else:
        return np.nan

transcript["offer_id"] = transcript["value"].apply(transform_offer_id)   
transcript["amount"] = transcript["value"].apply(transform_ammount)
transcript = transcript.drop("value", axis = 1)
    



In [328]:
transcript[transcript.event == "offer completed"]

,person,event,time,offer_id,amount
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,NaN,NaN
12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,0,NaN,NaN
12679,629fc02d56414d91bca360decdfa9288,offer completed,0,NaN,NaN
12692,676506bad68e4161b9bbaffeb039626b,offer completed,0,NaN,NaN
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,0,NaN,NaN
...,...,...,...,...,...
306475,0c027f5f34dd4b9eba0a25785c611273,offer completed,714,NaN,NaN
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,714,NaN,NaN
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,714,NaN,NaN
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,714,NaN,NaN


In [219]:
merged = profile.merge(transcript, right_on="person", left_on="id", how = "outer").drop("person", axis = 1)
offer_ids_type = {k: v for k, v in (zip(portfolio.id, portfolio.offer_type))}         

merged["offer_type"] = merged["offer_id"].map(offer_ids_type)
#merged = merged.drop("offer_id", axis =1)


#merged[merged["event"]!="transaction"].groupby(["id","event"])["event"]

merged = merged.sort_values(by="time")
merged = merged.merge(portfolio, left_on="offer_id", right_on ="id", how="outer").drop(["became_member_on","id_y",], axis=1)

In [262]:
test_ = merged.sort_values(by=["id_x", "time"], ascending =[True, True]).sort_values(by=["id_x","time"], ascending=[True, True])


unique_users = test_.id_x.unique()

unique_users

array(['0009655768c64bdeb2e877511632db8f',
       '00116118485d4dfda04fdbaba9a87b5c',
       '0011e0d4e6b944f998e987f904e8c1e5', ...,
       'fff8957ea8b240a6b5e634b6ee8eafcf',
       'fffad4f4828548d1b5583907f2e9906b',
       'ffff82501cea40309d5fdd7edcca4a07'], dtype=object)

In [284]:
#test[test["id_x"] == "0011e0d4e6b944f998e987f904e8c1e5"] 

test_

,gender,age,id_x,income,event,time,offer_id,amount,offer_type_x,reward,difficulty,duration,offer_type_y,web,email,mobile,social
44240,M,33,0009655768c64bdeb2e877511632db8f,72000.0,offer received,168,5a8bc65990b245e5a138643cd4eb9837,NaN,informational,0.0,0.0,3.0,informational,0.0,1.0,1.0,1.0
46101,M,33,0009655768c64bdeb2e877511632db8f,72000.0,offer viewed,192,5a8bc65990b245e5a138643cd4eb9837,NaN,informational,0.0,0.0,3.0,informational,0.0,1.0,1.0,1.0
153346,M,33,0009655768c64bdeb2e877511632db8f,72000.0,transaction,228,NaN,22.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60410,M,33,0009655768c64bdeb2e877511632db8f,72000.0,offer received,336,3f207df678b143eea3cee63160fa8bed,NaN,informational,0.0,0.0,4.0,informational,1.0,1.0,1.0,0.0
61786,M,33,0009655768c64bdeb2e877511632db8f,72000.0,offer viewed,372,3f207df678b143eea3cee63160fa8bed,NaN,informational,0.0,0.0,4.0,informational,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243350,F,45,ffff82501cea40309d5fdd7edcca4a07,62000.0,offer completed,576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291054,F,45,ffff82501cea40309d5fdd7edcca4a07,62000.0,offer received,576,2906b810c7d4411798c6938adc9daaa5,NaN,discount,2.0,10.0,7.0,discount,1.0,1.0,1.0,0.0
291612,F,45,ffff82501cea40309d5fdd7edcca4a07,62000.0,offer viewed,582,2906b810c7d4411798c6938adc9daaa5,NaN,discount,2.0,10.0,7.0,discount,1.0,1.0,1.0,0.0
252826,F,45,ffff82501cea40309d5fdd7edcca4a07,62000.0,transaction,606,NaN,10.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [285]:
l = ['offer received', 'offer viewed', 'transaction', 'offer completed']

trans_result = []

for user in unique_users:
    test = test_[test_["id_x"] == user]
    
    for label, col in test.iterrows():
        if col["event"] == "offer received":

            offer_id_ = test.loc[label,"offer_id"]
            offer_ammount =  test.loc[label,"difficulty"]


            val_window = int(col["duration"]+col["time"])
            events_window_df = test[(test["time"] >= col["time"]  ) & (test["time"] <= val_window) ]
            #display(events_window_df)

            #check if all relevant events happened
            events_window = events_window_df["event"].to_list()

            if not all(elem in events_window  for elem in l):
                continue

            else:

                trans = {}
                o_viewed = {}
                o_completed = []



                for label, row in events_window_df.iterrows():
                    if row["event"] == "transaction":
                        trans[label] = list([row["time"],row["amount"]])

                    elif row["event"] == "offer viewed":
                        o_viewed[label] = list([row["time"],row["offer_id"]])

                    elif row["event"] == "offer completed":  
                        o_completed.append(row["time"])


                #print(trans)
                #print(o_viewed)
                #print(o_completed)


                


                #print(label)
                for trans_ , values in trans.items():
                    time = values[0]
                    amount = values[1]

                    for offer_viewed, values in o_viewed.items():
                            time_viewed = values[0]
                            offer_id = values[1]

                            #print(offer_id)
                            #print(offer_id_)
                            #print(time_viewed)
                            #print(time)
                            #print(amount)
                            #print(offer_ammount)

                            if time_viewed <= time and offer_id == offer_id_ and time in o_completed and amount >= offer_ammount:
                                trans_result.append(trans_)
                                
                                #print(user+str(trans_result))
                                


KeyboardInterrupt: 

[192958,
 244032,
 176069,
 220396,
 109781,
 174018,
 223031,
 193064,
 173683,
 194411,
 174360,
 173815,
 108618,
 139026,
 109112,
 140170,
 242740,
 140857,
 140483,
 222241,
 242894,
 192849,
 139138,
 243506,
 245360,
 245845,
 242677,
 174383,
 195543,
 138584,
 109176,
 140619,
 220683,
 223643,
 175330,
 222981,
 195634,
 140427,
 109579,
 176048,
 221340,
 109400,
 173377,
 220713,
 139908,
 173918,
 242756,
 195272,
 108522,
 174781,
 173372,
 222480,
 138522,
 176050,
 192603,
 175199,
 108574,
 138990,
 221008,
 221600,
 173835,
 194099,
 176022,
 139317,
 108973,
 173462,
 107969,
 138844,
 191996,
 245411,
 107882,
 194328,
 140232,
 195598,
 176141,
 242387,
 109331,
 174365,
 221085,
 245171,
 140403,
 224049,
 195186,
 195049,
 242241,
 176243,
 193703,
 192749,
 139665,
 192543,
 195556,
 194337,
 223678,
 107899,
 223311,
 242406,
 139207,
 191900,
 107977,
 138999,
 140380,
 138794,
 175348,
 220772,
 193057,
 174337,
 108413,
 174840,
 243822,
 222513,
 243285,
 

,gender,age,id_x,income,event,time,offer_id,amount,offer_type_x,reward,difficulty,duration,offer_type_y,web,email,mobile,social
192958,F,62,0091d2b6a5ea4defaa8393e4e816db60,81000.0,transaction,408,NaN,27.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244032,O,52,00c20a9202d5475190b31a24de6fb06d,80000.0,transaction,576,NaN,16.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176069,F,22,00c91f31f5f74e769fa7a359b63e1a9f,49000.0,transaction,342,NaN,8.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220396,F,58,00d6dc87be4146ceb47fcd4baaaf6477,115000.0,transaction,504,NaN,39.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109781,M,57,00fac72fd6ad448e8019b175267023df,80000.0,transaction,6,NaN,26.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140004,M,85,cad006df49e5400b8b38069f296c97c7,66000.0,transaction,174,NaN,16.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242978,None,118,cafb4a8e15954efc83973d2607e5b66a,NaN,transaction,576,NaN,2.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139258,M,37,cb576b0ddc484c00afd56feb4fb63900,56000.0,transaction,168,NaN,11.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173483,M,37,cb576b0ddc484c00afd56feb4fb63900,56000.0,transaction,336,NaN,16.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dict = defaultdict(list)

NameError: name 'defaultdict' is not defined